In [31]:
import pandas as pd
import numpy as np
import re
import string

In [17]:
!pip install requests
!pip install urllib.parse

ERROR: Could not find a version that satisfies the requirement urllib.parse (from versions: none)
ERROR: No matching distribution found for urllib.parse


In [72]:
def remove_rt(string):
    return re.sub(r"^RT @\w ", "", string)

# From "@musicmadmarc @SocialDilemma_ @netflix @Facebook I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_beginning_mentions(string):
#     print(string)
#     if(re.search(r'@\w+',string) != None):
    # if(string =='@'):
    new_string=re.sub(r'^[@\w ]*','',string,1)
    new_string = new_string.strip()
    string = remove_beginning_mentions(new_string)
    return string

# From "#musicmadmarc #SocialDilemma_ #netflix #Facebook I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_beginning_hashs(string):
    # if(string[0]=='#'):
    new_string=re.sub(r'^[#\w ]*','',string,1)
    new_string = new_string.strip()
    string = remove_beginning_mentions(new_string)
    return string

# From " I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K @musicmadmarc @SocialDilemma_ @netflix @Facebook"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_last_mentions(string):
    last_mention = string.rfind('@')
    last_space = string.rfind(' ')
    if(last_mention > last_space):
        string = string[:last_space]
        string = remove_last_mentions(string)
    return string

# From " I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K @musicmadmarc @SocialDilemma_ @netflix @Facebook"
# to: "I'm also reminded of the very poignant quote by French philosopher… https://t.co/CA52aepW6K"

def remove_last_hashs(string):
    last_mention = string.rfind('#')
    last_space = string.rfind(' ')
    if(last_mention > last_space):
        string = string[:last_space]
        string = remove_last_mentions(string)
    return string

def remove_punctuation_notation(string):
    string = re.sub(r'[\$%&\'()*+\-\/=#@\[\\\]^_`{|}~]*','',string)
    return string

def preprocessing_pipeline(string):
    string = remove_rt(string)
    string = remove_beginning_mentions(string)
    string = remove_beginning_hashs(string)
    string = remove_last_mentions(string)
    string = remove_last_hashs(string)
    string = remove_punctuation_notation(string)
    return string


In [4]:
mask = pd.read_csv('./output/mask_text.csv')
vaccine = pd.read_csv('./output/vaccine_text.csv')

C:\Users\aruba\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\aruba\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (7,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [89]:
import requests
from urllib.parse import urlsplit

# Input the t.co format url and return the original link
def to_source_url(shorturl):
    try:
        r = requests.get(shorturl)
        src = urlsplit(r.url)[1]
        return src
    except:
        return None

def find_url(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    return links

In [29]:
text = list(vaccine['text'])

In [81]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def strip_all_entities(text):
    entity_prefixes = ['@','#','RT @']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

tests = [
    "@peter I really love that shirt at #Macy. http://bet.ly//WjdiW4",
    "@shawn Titanic tragedy could have been prevented Economic Times: Telegraph.co.ukTitanic tragedy could have been preve... http://bet.ly/tuN2wx",
    "I am at Starbucks http://4sh.com/samqUI (7419 3rd ave, at 75th, Brooklyn)",
]
for t in tests:
    print(strip_all_entities(strip_links(t)))

I really love that shirt at
Titanic tragedy could have been prevented Economic Times Telegraph co ukTitanic tragedy could have been preve
I am at Starbucks 7419 3rd ave at 75th Brooklyn


In [84]:
text = list(vaccine['text'])
filtered = []
urls = []
for t in text:
    result = find_url(t)
    url_list = []
    for r in result:
        url_list.append(r[0])
    urls.append(url_list)    
    cleared_text = strip_links(t)
    cleared_text = strip_all_entities(cleared_text)
    filtered.append(cleared_text)

In [86]:
vaccine['filtered text'] = filtered

In [ ]:
src_url = []
for url_list in urls:
    src_list = []
    for url in url_list:
        src_list.append(to_source_url(url))
    src_url.append(src_list)

In [ ]:
src_url

In [65]:
vaccine['src_url'] = src_url

In [ ]:
vaccine.to_csv('./output/filtered_vaccine_text.csv',index = False)